Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Search context from structured data store on Vertex AI Search

# Configuration
## Install python packages


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform \
                               google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 34.4 MB/s eta 0:00:00


## Authentication to access to the GCP
* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [2]:
import sys
from IPython.display import Markdown, display

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

!gcloud config set project ai-hangsik

Updated property [core/project].


## Set the environment on GCP Project


In [3]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Tool,
)

from vertexai.preview.generative_models import grounding

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [5]:
SEARCH_APP_LOCATION = "global"
SEARCH_ENGINE_ID = "layout-parser_1718022685327"

### Search configuration

In [6]:
def search_context(query:str):

  from google.api_core.client_options import ClientOptions
  # from google.cloud import discoveryengine_v1alpha as discoveryengine
  from google.cloud import discoveryengine

  # Create a client using a regional endpoint
  client = discoveryengine.SearchServiceClient(
      client_options=(
          ClientOptions(
              api_endpoint=f"{SEARCH_APP_LOCATION}-discoveryengine.googleapis.com"
          )
          if SEARCH_APP_LOCATION != "global"
          else None
      )
  )

  # The full resource name of the search app serving config
  serving_config = f"projects/{PROJECT_ID}/locations/{SEARCH_APP_LOCATION}/collections/default_collection/engines/{SEARCH_ENGINE_ID}/servingConfigs/default_config"

  # [ContentSearchSpec] https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
  content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(

      # [SnippetSpec] https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec.SnippetSpec
      snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
          return_snippet=False
      ),

      # [SummarySpec] https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec.SummarySpec
      summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
          summary_result_count=5,
          include_citations=False,
          ignore_adversarial_query=True,
          ignore_non_summary_seeking_query=True,
          model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
              preamble="Summarize the contents" # Text at the beginning of the prompt that instructs the assistant. Examples are available in the user guide.
          ),
          model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
              version="stable",
          ),
      ),

      # [ExtractiveContentSpec] https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec
      extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
          max_extractive_segment_count=1,
          # max_extractive_answer_count = 5, # Not permitted for chunking(Layout parser)
          return_extractive_segment_score= True,
          num_previous_segments = 1,
          num_next_segments = 1
      ),
    )

  # Refer to the `SearchRequest` reference for all supported fields:
  # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
  request = discoveryengine.SearchRequest(
      serving_config=serving_config,
      query=query,
      page_size=10,
      content_search_spec=content_search_spec,
      query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
          condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
      ),
      spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
          mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
      ),
  )

  page_results = client.search(request)

  return page_results


### contents parsing

In [7]:

def get_contents(response):
  """Parse response to build a conext to be sent to LLM
     Parsing Unstructured datastore.
  """

  import json

  retrieved_data: list[dict] = []

  for result in response:

    for key, value in result.document.derived_struct_data.items():
      if key =="extractive_segments":
        for segment in value:

          dict_segment = {}

          for key, value in segment.items():

            if key =="relevanceScore":
              dict_segment['relevanceScore']=value

            if key == "content":
              dict_segment['content']=value

          retrieved_data.append(dict_segment)

  return  retrieved_data

## Reasoning result with LLM

### Function to call LLM

In [8]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        stream=False,
    )

    return responses.text

### Run example

In [9]:
from time import perf_counter

t1_start = perf_counter()

query = " 개인정보의 수집, 이용, 제공 대해서 설명해주세요. "

result = search_context(query)

print(result)

retrieved_data = get_contents(result)

for context in retrieved_data:
  print(context)


prompt = f"""

  당신은 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 내용을 참고해서 답변해주세요.
  Context : {retrieved_data}
  Question : {query}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


SearchPager<results {
  id: "1"
  document {
    struct_data {
      fields {
        key: "title"
        value {
          string_value: "개인정보 보호법(법률)(제19234호)(20240315)"
        }
      }
      fields {
        key: "category"
        value {
          list_value {
            values {
              string_value: "개인정보 보호법"
            }
            values {
              string_value: "개인정보"
            }
          }
        }
      }
    }
    name: "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/branches/0/documents/1"
    id: "1"
    derived_struct_data {
      fields {
        key: "link"
        value {
          string_value: "gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf"
        }
      }
      fields {
        key: "extractive_segments"
        value {
          list_value {
            values {
              struct_value {
                fields {
                  key

## 개인정보의 수집, 이용, 제공에 대한 설명입니다.

**개인정보보호법 제15조(개인정보의 수집·이용)**에 따르면 개인정보처리자는 다음과 같은 경우에만 개인정보를 수집하고 이용할 수 있습니다.

1. **정보주체의 동의를 받은 경우:** 개인정보처리자는 정보주체의 동의를 얻어 개인정보를 수집하고 이용할 수 있습니다. 이 때, 정보주체에게 수집·이용 목적, 개인정보 항목, 보유 및 이용 기간, 동의 거부 권리 및 불이익 내용 등을 알려야 합니다.
2. **법률에 특별한 규정이 있거나 법령상 의무를 준수하기 위하여 불가피한 경우:** 법률에서 개인정보 수집·이용을 허용하거나, 법령상 의무를 이행하기 위해 개인정보 수집·이용이 불가피한 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다.
3. **공공기관이 법령 등에서 정하는 소관 업무의 수행을 위하여 불가피한 경우:** 공공기관이 법령 등에서 정한 소관 업무를 수행하기 위해 개인정보 수집·이용이 불가피한 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다.
4. **정보주체와 체결한 계약을 이행하거나 계약을 체결하는 과정에서 정보주체의 요청에 따른 조치를 이행하기 위하여 필요한 경우:** 정보주체와 계약을 이행하거나 계약 체결 과정에서 정보주체의 요청에 따른 조치를 이행하기 위해 개인정보 수집·이용이 필요한 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다.
5. **명백히 정보주체 또는 제3자의 급박한 생명, 신체, 재산의 이익을 위하여 필요하다고 인정되는 경우:** 정보주체 또는 제3자의 급박한 생명, 신체, 재산의 이익을 위해 개인정보 수집·이용이 필요하다고 인정되는 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다.
6. **개인정보처리자의 정당한 이익을 달성하기 위하여 필요한 경우로서 명백하게 정보주체의 권리보다 우선하는 경우:** 개인정보처리자의 정당한 이익을 달성하기 위해 개인정보 수집·이용이 필요하고, 정보주체의 권리보다 우선하는 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다. 이 경우 개인정보처리자의 정당한 이익과 상당한 관련이 있고 합리적인 범위를 초과하지 않아야 합니다.
7. **공중위생 등 공공의 안전과 안녕을 위하여 긴급히 필요한 경우:** 공중위생 등 공공의 안전과 안녕을 위해 개인정보 수집·이용이 긴급히 필요한 경우에는 정보주체의 동의 없이 개인정보를 수집·이용할 수 있습니다.

**개인정보보호법 제17조(개인정보의 제공)**에 따르면 개인정보처리자는 다음과 같은 경우에만 정보주체의 개인정보를 제3자에게 제공할 수 있습니다.

1. **정보주체의 동의를 받은 경우:** 개인정보처리자는 정보주체의 동의를 얻어 개인정보를 제3자에게 제공할 수 있습니다. 이 때, 정보주체에게 개인정보를 제공받는 자, 개인정보 이용 목적, 제공하는 개인정보 항목, 개인정보 보유 및 이용 기간, 동의 거부 권리 및 불이익 내용 등을 알려야 합니다.
2. **제15조제1항제2호, 제3호 및 제5호부터 제7호까지에 따라 개인정보를 수집한 목적 범위에서 개인정보를 제공하는 경우:** 법률상 의무, 공공기관의 업무 수행, 급박한 위험, 정당한 이익 등의 목적으로 개인정보를 수집한 경우에는 그 목적 범위 내에서 개인정보를 제3자에게 제공할 수 있습니다.

**개인정보보호법 제16조(개인정보의 수집 제한)**에 따르면 개인정보처리자는 개인정보를 수집할 경우, 목적에 필요한 최소한의 개인정보만 수집해야 합니다. 이 경우 최소한의 개인정보 수집에 대한 입증 책임은 개인정보처리자가 부담합니다.

**요약하자면, 개인정보 수집, 이용, 제공은 정보주체의 동의를 기반으로 하며, 법률에서 허용하는 경우에만 정보주체의 동의 없이 가능합니다. 또한, 수집 및 이용 목적 범위 내에서만 가능하며, 최소한의 정보만 수집해야 합니다.**

**주의:** 위 내용은 개인정보보호법의 일부 조항을 간략하게 설명한 것입니다. 개인정보보호법 전문을 참고하여 자세한 내용을 확인하시기 바랍니다.
